# License Plate Recognition Project


This notebook implements the entire pipeline for license plate detection and recognition, including:
1. **Data Loading and Preprocessing**  
2. **YOLOv8 Model Training for License Plate Detection**  
3. **OCR Integration for Character Recognition**  
4. **End-to-End Pipeline Execution**  
5. **Evaluation and Metrics**  

Detailed comments are provided to help explain each part of the code.


## Step 1: Importing Necessary Libraries

In [ ]:

# Import essential libraries
import os
import cv2
import numpy as np
from ultralytics import YOLO
import easyocr
from sklearn.metrics import accuracy_score
    

## Step 2: Load and Preprocess Datasets

In [ ]:

# Define paths to datasets
training_data_1_path = "path_to_training_data_1"  # Car images with bounding boxes
training_data_2_path = "path_to_training_data_2"  # Number plate images with text
test_data_path = "path_to_test_data"  # Test set

# Load annotations and images
def load_data(path):
    images = []
    annotations = []
    for file in os.listdir(path):
        if file.endswith(".jpg") or file.endswith(".png"):
            images.append(cv2.imread(os.path.join(path, file)))
        elif file.endswith(".txt"):
            with open(os.path.join(path, file), 'r') as f:
                annotations.append(f.read())
    return images, annotations

# Load data for each set
train1_images, train1_annotations = load_data(training_data_1_path)
train2_images, train2_annotations = load_data(training_data_2_path)
test_images, test_annotations = load_data(test_data_path)
    

## Step 3: Train YOLOv8 for License Plate Detection

In [ ]:

# Initializing YOLOv8 model
model = YOLO('yolov8n.pt')  # Using a pre-trained YOLOv8 nano model

# Preparing training configuration
# Training Set 1 is formatted in YOLO's expected structure with 'data.yaml'
data_yaml_path = "path_to_yolo_config.yaml"

# Train YOLO model
model.train(data=data_yaml_path, epochs=100, imgsz=640, name="license_plate_detection")
    

## Step 4: Use EasyOCR for Character Recognition

In [ ]:

# Initializing EasyOCR model
reader = easyocr.Reader(['en'])

# Function to recognize text from an image
def recognize_text(image_path):
    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    results = reader.readtext(image)
    return ''.join([res[1] for res in results])  # Combining all recognized strings
    

## Step 5: Detection and Recognition Pipeline

In [ ]:

# Combining YOLOv8 and OCR for end-to-end processing
def detect_and_recognize(image):
    # Detect license plates
    results = model(image)
    boxes = results[0].boxes.xyxy  # Extracting bounding box coordinates

    # Croping detected license plates and recognizing text
    recognized_texts = []
    for box in boxes:
        xmin, ymin, xmax, ymax = map(int, box)
        cropped = image[ymin:ymax, xmin:xmax]
        text = recognize_text(cropped)
        recognized_texts.append(text)

    return recognized_texts

# Running pipeline on test images
test_results = {}
for idx, test_image in enumerate(test_images):
    test_results[idx] = detect_and_recognize(test_image)
    

## Step 6: Evaluate the Model on the Test Set

In [ ]:

# Extracting ground-truth annotations for test set
ground_truths = [ann.split('\n') for ann in test_annotations]

# Comparing with predictions and calculating accuracy
all_predictions = []
all_ground_truths = []

for idx in range(len(test_images)):
    all_predictions.extend(test_results[idx])  # Flatten predictions
    all_ground_truths.extend(ground_truths[idx])  # Flatten ground truths

# Calculate accuracy
accuracy = accuracy_score(all_ground_truths, all_predictions)
print(f"License Plate Recognition Accuracy: {accuracy * 100:.2f}%")
    

## Results and Conclusion


The pipeline successfully combines YOLOv8 for license plate detection and EasyOCR for character recognition.
The achieved accuracy on the test set is reported above.  
Further improvements can be made by fine-tuning YOLOv8 and improving OCR integration.
